## Interferometry for dummies notebook

This notebook should take you through some of the concepts that have been discussed in the lecture. Therefore, clues to all the solutions should be present in the slides. The purpose here is to get some hands-on experience, not to get hung up on coding issues. If you are stuck, there is an equivalent notebook containing the solutions from which you can draw inspiration.

In [2]:
import numpy as np
from copy import copy

import matplotlib.pyplot as plt
from astropy.convolution import convolve, Gaussian2DKernel
from astropy.io import ascii, fits

### Fourier transforms in 1D

To get a feel for the fast Fourier transform functionality in numpy, see if you can get the primary beam pattern of single dish, assuming the aperture is uniformly illuminated.

As the Fourier transform is discrete and finite, the resulting function will not be perfect. Check to see how the size of the aperture relative to the sampled space and the sampling interval influence the result.

**If your results are strange or unexpected**: Whereas we like our signals to be in the middle of the sampled space for aesthetic reasons, the discrete Fourier transform follows the python indexing, which means that the 0-point will be right at the edges of the array. In general, `np.fft.fftshift` is your best friend here, and it is best to use it before and after every Fourier transform. Trust me, this will save you a lot of trouble.

### Simulating MeerKAT observations

In the data subdirectory, a text file is included with the UV-coverage of a MeerKAT observation. 

1. Start with plotting the UV-coverage, preferably in units of $k\lambda$, assuming $\lambda=$21 cm.

2. Using 2D Fourier transforms, convert the UV-coverage to a Point-spread-function (PSF). To do this you need to define a grid for UV space first, which will define the resolution and size of your resulting image. **Bonus**: Implement both natural and uniform weighting schemes, and see how this influences the shape of the PSF. 

3. In the data subdirectory there is a fits file containing the sky emission from a FRII radio galaxy. Using the data just obtained, convolve this image with PSF using 2D Fourier transforms.

In [ ]:
#Load in MeerKAT UV-coverage
meerkat_uv = ascii.read('data/MeerKAT_uv_coverage.txt')

In [ ]:
def uv_points_to_psf(u, v, imsize, weighting):
    '''
    Grid and Fourier transform UV coverage to create a PSF
    '''

In [ ]:
# Load in radio galaxy fits file
radio_galaxy = fits.open('data/radio_galaxy.fits')[0].data

### Implementing CLEAN

In the data subdirectory you will find a dirty image containing a number of point sources. In the same directory an image of the PSF is included. 

To clean the image, implement the Högbom CLEAN algorithm as specified in the slides, including a gain factor, an iteration limit, and a noise treshold. To create the clean image, the 2D Gaussian fit to the central lobe of the beam is 11x7.8 pixels with PA of -110, I have already defined as an astropy kernel. I have also included a function which takes care of subtracting two images which are shifted w.r.t. each other.

In [ ]:
# Conversion factor from FWHM to std
a = 2*np.sqrt(2*np.log(2))

maj_std = 11/a
min_std = 7.8/a
theta = np.deg2rad(-110)

# Define clean beam
clean_beam = Gaussian2DKernel(x_stddev = min_std,
                              y_stddev = maj_std,
                              theta=theta,
                              x_size=51, y_size=51)
# Normalize the beam
clean_beam_normed = clean_beam.array/clean_beam.array.max()

In [ ]:
def subtract_shifted_images(im1, im2, X, Y, amp):
    '''
    Subtract amp*im2 from im1, given an offset in X and Y
    '''
    im1_xshape = int(im1.shape[0]/2)
    im1_yshape = int(im1.shape[1]/2)
        
    im2_xshape = int(im2.shape[0])
    im2_yshape = int(im2.shape[1])
    
    minY = min(int(Y)-im1_yshape, 0)
    minX = min(int(X)-im1_xshape, 0)
    maxY = max(int(Y)+im1_yshape-im2_yshape, 0)
    maxX = max(int(X)+im1_xshape-im2_xshape, 0)

    im1[int(X)-im1_xshape-minX:int(X)+im1_xshape-maxX,
        int(Y)-im1_yshape-minY:int(Y)+im1_yshape-maxY] -= amp*im2[-minX:im2_xshape-maxX,-minY:im2_yshape-maxY]
    
    return im1

In [ ]:
def hogbom_clean(dirty_image, psf, gain, niter, thresh):
    '''
    Implement Hogbom clean with gain, niter and threshold
    '''

In [ ]:
# Load in dirty image and PSF
dirty_image = fits.open('data/dirty_image.fits')[0].data
psf = fits.open('data/psf.fits')[0].data